- [X] see sql5 file
- [X] variable match and selection
- [ ] define inclusion-exclusion criteria
- [ ] define target trial (cite hernan)

# All the features (Slide 11, page 14)

### Demographic:
- Gender *
- Age (IQR) *
- Ethnicity (Race: White, black, multiple or others) *
### Diseases:
- Other diseases (icd9 codes) *
- Surgery *
- Pneumonia *
- Infection
- Sepsis (diagnosis at admission) *
- Dementia *
- Alzheimer's *
- Depression *
- Anxiety *
- Coma (drug induced, miscellaneous, combination)
- Metabolic acidosis
- Respiratory failure
- ARDS (diagnosis at admission)
- Airway Protection (diagnosis at admission)
- Chronic Obstructive Pulmonary Disease, Asthma, or other pulmonary disorder (diagnosis at admission)
- Chronic heart failure, myocardial infarction, or arrhythmia (diagnosis at admission)
- Cirrhosis or Liver failure (diagnosis at admission)
- Other (diagnosis at admission)
### Drugs:
- List of drugs prescribed *
- Received APD treatment - no (%) (before admission, before admission and randomization)
- APD prescribed *
- Distinct APD categories *
### Admission events:
- Admission category (surgical, medical, trauma, neurology/neurosurgical)
- Length of stay *
- Mechanical ventilation ^*
- Mechanical ventilation hours ^ *
- Sedatives use (yes)
- Morphine use (mg/day)
- Urea concentration (mmol/L)
- Urgent admission (%)
- Hx of cognitive impairment
- Hx of ETOH use 
- MAP (mean arterial blood pressure) at the time of ICU admission
- Use of Corticosteroids
- BUN (blood urea nitrogen) at the of ICU admission
- Admitted to Surgical ICU - no (%)
- Received assisted ventilation before randomization - no (%) Invasive and Noninvasive
- Shock before randomization - no (%)
- Median no of days from ICU admission to randomization - (IQR)
### Evaluation:
- APACHE-II score (Acute Physiology & Chronic Health Evaluation) (at randomization)
 - APSIII in place of Apache score https://github.com/MIT-LCP/mimic-code/issues/653
- Median Short-form IQCODE score (IQR)
- Median Charlson Comorbidity Index Score (IQR)
- Median SOFA score at randomization
- Hyperactive Delirium at Randomization- no (%)
- Hypoactive Delirium at Randomization - no (%)
### Outcomes:
- Death in hospital *
- Death in one year *

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv("delirium_data.csv")

In [4]:
data.shape

(1671, 30)

In [5]:
data.columns

Index(['subject_id', 'gender', 'age', 'ethnicity', 'hadm_id', 'icustay_id',
       'los_days', 'icd9_codes', 'sofa', 'apsiii', 'mechvent_starttime',
       'mechvent', 'mechvent_duration_hours', 'drugs', 'dist_cat_count',
       'drug_categories', 'had_surgery', 'had_pneu', 'sepsis', 'dementia',
       'alzheimers', 'depression', 'anxiety', 'death_hosp', 'death_one_year',
       'drugstarttime', 'admittime', 'dischtime', 'deathtime',
       'patient_deathtime'],
      dtype='object')

# Deleted deathtime
- ADMISSIONS.DEATHTIME (https://mimic.mit.edu/docs/iii/tables/admissions/)
 - If applicable, DEATHTIME provides the time of in-hospital death for the patient. Note that DEATHTIME is only present if the patient died in-hospital, and is almost always the same as the patient’s DISCHTIME. However, there can be some discrepancies due to typographical errors.
 - Use to find death in hospital

- PATIENTS.DOD (https://mimic.mit.edu/docs/iii/tables/patients/)
 - DOD is the date of death for the given patient. DOD_HOSP is the date of death as recorded in the hospital database. DOD_SSN is the date of death from the social security database. Note that DOD merged together DOD_HOSP and DOD_SSN, giving priority to DOD_HOSP if both were recorded.
 - Real death time
 
- Just DOD works fine.

In [6]:
data = data.drop('deathtime', 1)

data = data.rename(columns={
    "dist_cat_count": "drug_categories_distinct_count", 
    "gender": "sex", 
    "patient_deathtime": "deathtime", 
    "mechvent": "mechvent_count", 
    "had_surgery": "surgery", 
    "had_pneu": "pneumonia"})

/var/folders/k5/t6qrq2v55gxd80p9mdn45x9r8q8tz2/T/ipykernel_22719/3729729959.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('deathtime', 1)


In [7]:
data.shape

(1671, 29)

In [8]:
data.columns

Index(['subject_id', 'sex', 'age', 'ethnicity', 'hadm_id', 'icustay_id',
       'los_days', 'icd9_codes', 'sofa', 'apsiii', 'mechvent_starttime',
       'mechvent_count', 'mechvent_duration_hours', 'drugs',
       'drug_categories_distinct_count', 'drug_categories', 'surgery',
       'pneumonia', 'sepsis', 'dementia', 'alzheimers', 'depression',
       'anxiety', 'death_hosp', 'death_one_year', 'drugstarttime', 'admittime',
       'dischtime', 'deathtime'],
      dtype='object')

In [9]:
data.sofa = data.sofa.fillna(-1)
data.apsiii = data.apsiii.fillna(-1)

In [10]:
data.icustay_id = data.icustay_id.fillna(0)
data.surgery = data.surgery.fillna(2)
data = data.astype({"icustay_id": int, "sofa": int, "apsiii": int, "sex": 'category', "ethnicity": 'category', "surgery": int})

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   subject_id                      1671 non-null   int64   
 1   sex                             1671 non-null   category
 2   age                             1671 non-null   float64 
 3   ethnicity                       1671 non-null   category
 4   hadm_id                         1671 non-null   int64   
 5   icustay_id                      1671 non-null   int64   
 6   los_days                        1665 non-null   float64 
 7   icd9_codes                      1671 non-null   object  
 8   sofa                            1671 non-null   int64   
 9   apsiii                          1671 non-null   int64   
 10  mechvent_starttime              854 non-null    object  
 11  mechvent_count                  1671 non-null   int64   
 12  mechvent_duration_ho

In [12]:
data.sample(10)

,subject_id,sex,age,ethnicity,hadm_id,icustay_id,los_days,icd9_codes,sofa,apsiii,mechvent_starttime,mechvent_count,mechvent_duration_hours,drugs,drug_categories_distinct_count,drug_categories,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,death_hosp,death_one_year,drugstarttime,admittime,dischtime,deathtime
439,22437,M,300.00,WHITE,111823,253867,2.9688,42789: Cardiac dysrhythmias NEC\n7210: Cervica...,4,55,NaN,0,NaN,No drug,0,NO DRUG,2,0,0,0,0,0,0,0,1,2179-11-25 19:52:00,2179-11-25 19:50:00,2179-12-03 13:33:00,2179-12-31 00:00:00
566,27561,F,77.42,BLACK/AFRICAN AMERICAN,153158,205063,4.8398,0389: Septicemia NOS\n4280: CHF NOS\n486: Pneu...,8,99,NaN,0,NaN,No drug,0,NO DRUG,1,1,1,0,0,0,0,0,0,2161-07-01 22:30:08,2161-07-01 20:08:00,2161-07-09 17:30:00,NaN
1127,64191,M,69.69,WHITE,186864,252380,4.8505,0389: Septicemia NOS\n78552: Septic shock\n410...,9,45,NaN,0,NaN,Quetiapine Fumarate\nHaloperidol\nHaloperidol\...,3,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE,0,0,1,0,0,0,0,0,0,2102-04-21 22:09:03,2102-04-21 22:08:00,2102-05-03 16:42:00,NaN
1000,55730,M,62.02,WHITE,195438,245211,23.9861,430: Subarachnoid hemorrhage\n74781: Cerebrova...,2,20,2113-03-29 18:00:00,2,"19.2500000000000000, 1.3333333333333333",Quetiapine Fumarate\nQuetiapine Fumarate\nQuet...,1,QUETIAPINE,1,0,0,0,0,0,0,0,0,2113-03-29 16:16:11,2113-03-29 16:15:00,2113-04-25 17:37:00,NaN
1397,81412,F,88.59,WHITE,193352,278624,4.6487,47832: Vocal paral unilat total\n5070: Food/vo...,2,40,NaN,0,NaN,No drug,0,NO DRUG,0,0,0,0,0,0,0,0,1,2190-07-10 23:53:01,2190-07-10 23:51:00,2190-07-22 15:15:00,2190-07-27 00:00:00
306,15358,M,57.62,WHITE,105688,285673,0.7914,5990: Urin tract infection NOS\n99831: Disrup ...,2,22,NaN,0,NaN,Prochlorperazine\nProchlorperazine,1,PROCHLORPERAZINE,0,1,0,0,0,0,0,0,1,2142-05-08 23:06:18,2142-05-07 14:00:00,2142-07-14 15:50:00,2142-08-25 00:00:00
1077,61144,F,51.83,WHITE,187912,280981,4.9400,8082: Fracture of pubis-closed\n51851: Ac resp...,2,33,2189-07-08 22:00:00,5,"36.2500000000000000, 7.0000000000000000, 16.33...",Haloperidol\nQuetiapine Fumarate\nHaloperidol\...,2,HALOPERIDOL\nQUETIAPINE,1,0,0,0,0,0,0,0,0,2189-07-08 19:40:19,2189-07-08 19:39:00,2189-07-18 12:50:00,NaN
1347,77471,M,26.52,ASIAN - CAMBODIAN,112557,269077,0.3669,23875: Myelodysplastic synd NOS\n4940: Bronchi...,4,28,NaN,0,NaN,No drug,0,NO DRUG,0,0,1,0,0,0,0,0,1,2138-04-02 16:02:23,2138-03-29 12:43:00,2138-04-11 14:45:00,2138-08-19 00:00:00
1128,64357,M,80.48,WHITE,197694,272943,1.6159,5570: Ac vasc insuff intestine\n5856: End stag...,6,47,NaN,0,NaN,Haloperidol\nHaloperidol\nHaloperidol\nQuetiap...,2,HALOPERIDOL\nQUETIAPINE,0,0,0,0,0,0,0,0,1,2105-01-03 20:03:21,2105-01-02 07:15:00,2105-01-09 14:50:00,2105-08-07 00:00:00
909,50404,M,29.87,WHITE,110630,228881,12.1507,04119: Other staphylococcus\n99663: React-nerv...,0,22,2110-10-05 09:00:00,1,168.5000000000000000,Prochlorperazine\nProchlorperazine\nQuetiapine...,2,PROCHLORPERAZINE\nQUETIAPINE,1,0,1,0,0,0,0,0,0,2110-10-01 17:07:49,2110-09-21 22:40:00,2110-10-25 15:48:00,NaN


# Checking for unique values and resolving if necessary

In [13]:
print(len(data.subject_id.unique()))
print(len(data.hadm_id.unique()))
print(len(data.icustay_id.unique()))

1398
1445
1666


In [14]:
print("number of cases:", data[data.icustay_id.duplicated(keep=False)].shape)
data[data.icustay_id.duplicated(keep=False)]

number of cases: (6, 29)


,subject_id,sex,age,ethnicity,hadm_id,icustay_id,los_days,icd9_codes,sofa,apsiii,mechvent_starttime,mechvent_count,mechvent_duration_hours,drugs,drug_categories_distinct_count,drug_categories,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,death_hosp,death_one_year,drugstarttime,admittime,dischtime,deathtime
82,3734,F,57.12,WHITE,154629,0,NaN,25070: DMII circ nt st uncntrld\n44024: Ath ex...,-1,-1,NaN,0,NaN,No drug,0,NO DRUG,2,0,0,0,0,0,0,0,0,NaN,2110-08-26 00:08:00,2110-08-30 16:36:00,2111-12-04 00:00:00
144,6537,F,82.79,WHITE,152603,0,NaN,3485: Cerebral edema\n1912: Mal neo temporal l...,-1,-1,NaN,0,NaN,Quetiapine Fumarate\nQuetiapine Fumarate\nOlan...,2,OLANZAPINE\nQUETIAPINE,1,0,0,0,0,0,0,0,1,NaN,2128-07-30 23:20:00,2128-08-05 14:00:00,2128-09-21 00:00:00
178,8450,M,79.10,WHITE,157731,0,NaN,4414: Abdom aortic aneurysm\n4422: Iliac arter...,-1,-1,NaN,0,NaN,Haloperidol,1,HALOPERIDOL,0,0,0,0,0,0,0,0,0,NaN,2112-09-12 15:48:00,2112-09-19 16:11:00,NaN
203,9795,M,77.68,BLACK/AFRICAN AMERICAN,182522,0,NaN,99665: React-oth genitourin dev\n6084: Male ge...,-1,-1,NaN,0,NaN,Haloperidol\nHaloperidol\nHaloperidol\nHaloper...,1,HALOPERIDOL,0,0,0,0,0,0,0,0,0,NaN,2105-10-26 21:57:00,2105-11-03 15:24:00,2109-04-26 00:00:00
300,15047,F,73.89,WHITE,137209,0,NaN,57451: Choledochlith NOS w obst\n99679: Comp-i...,-1,-1,NaN,0,NaN,No drug,0,NO DRUG,2,0,0,0,0,0,0,0,0,NaN,2126-04-03 08:00:00,2126-04-09 11:53:00,2127-06-30 00:00:00
1295,75054,M,48.46,WHITE,170546,0,NaN,73819: Oth spcf deformity head\n2930: Delirium...,-1,-1,NaN,0,NaN,Haloperidol\nProchlorperazine\nQuetiapine Fuma...,4,HALOPERIDOL\nPROCHLORPERAZINE\nQUETIAPINE\nZIP...,1,0,0,0,0,0,0,0,0,NaN,2159-12-12 12:15:00,2159-12-17 21:05:00,NaN


Seems like they are six unique icu stay case, just with icustay_id missing, so we can ignore this issue now.

In [15]:
print("number of cases:", data[data.hadm_id.duplicated(keep=False)].shape)
data[data.hadm_id.duplicated(keep=False)].head(5)

number of cases: (412, 29)


,subject_id,sex,age,ethnicity,hadm_id,icustay_id,los_days,icd9_codes,sofa,apsiii,mechvent_starttime,mechvent_count,mechvent_duration_hours,drugs,drug_categories_distinct_count,drug_categories,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,death_hosp,death_one_year,drugstarttime,admittime,dischtime,deathtime
1,214,M,63.44,WHITE,197273,200066,2.7044,"41071: Subendo infarct, initial\n4280: CHF NOS...",6,58,NaN,0,NaN,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-06 23:56:53,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
2,214,M,63.44,WHITE,197273,255525,3.0368,"41071: Subendo infarct, initial\n4280: CHF NOS...",4,49,2188-10-24 16:40:00,1,3.3333333333333333,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-24 15:45:55,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
3,214,M,63.44,WHITE,197273,241941,5.5964,"41071: Subendo infarct, initial\n4280: CHF NOS...",6,73,NaN,0,NaN,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-28 21:26:20,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
4,223,M,67.51,WHITE,105694,255617,15.8837,0389: Septicemia NOS\n41021: AMI inferolateral...,7,48,2157-05-01 06:00:00,5,"28.0000000000000000, 21.7500000000000000, 20.0...",Haloperidol\nHaloperidol\nHaloperidol\nHaloper...,2,HALOPERIDOL\nOLANZAPINE,1,1,1,0,0,0,0,0,0,2157-05-01 05:20:41,2157-05-01 05:19:00,2157-07-01 10:20:00,NaN
5,223,M,67.51,WHITE,105694,231395,4.0629,0389: Septicemia NOS\n41021: AMI inferolateral...,3,33,2157-06-01 15:55:00,1,11.5000000000000000,Haloperidol\nHaloperidol\nHaloperidol\nHaloper...,2,HALOPERIDOL\nOLANZAPINE,1,1,1,0,0,0,0,0,0,2157-06-01 11:33:51,2157-05-01 05:19:00,2157-07-01 10:20:00,NaN


In [16]:
temp = data[data.hadm_id.duplicated(keep=False)]
for i in temp.subject_id.unique():
    temp2 = temp[temp.subject_id == i]
    if (temp2.shape[0] != len(temp2.icustay_id.unique())): # check for unique icustay_id for each individual
        print(i)

They all have unique icustay_id, so treating as individual persons

In [17]:
print("number of cases:", data[data.subject_id.duplicated(keep=False)].shape)
data[data.subject_id.duplicated(keep=False)].head(5)

number of cases: (494, 29)


,subject_id,sex,age,ethnicity,hadm_id,icustay_id,los_days,icd9_codes,sofa,apsiii,mechvent_starttime,mechvent_count,mechvent_duration_hours,drugs,drug_categories_distinct_count,drug_categories,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,death_hosp,death_one_year,drugstarttime,admittime,dischtime,deathtime
1,214,M,63.44,WHITE,197273,200066,2.7044,"41071: Subendo infarct, initial\n4280: CHF NOS...",6,58,NaN,0,NaN,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-06 23:56:53,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
2,214,M,63.44,WHITE,197273,255525,3.0368,"41071: Subendo infarct, initial\n4280: CHF NOS...",4,49,2188-10-24 16:40:00,1,3.3333333333333333,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-24 15:45:55,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
3,214,M,63.44,WHITE,197273,241941,5.5964,"41071: Subendo infarct, initial\n4280: CHF NOS...",6,73,NaN,0,NaN,Haloperidol\nOlanzapine (Disintergrating Table...,4,HALOPERIDOL\nOLANZAPINE\nQUETIAPINE\nRISPERIDONE,0,0,0,0,0,0,0,0,0,2188-10-28 21:26:20,2188-10-06 23:56:00,2188-11-04 11:40:00,2195-07-13 00:00:00
4,223,M,67.51,WHITE,105694,255617,15.8837,0389: Septicemia NOS\n41021: AMI inferolateral...,7,48,2157-05-01 06:00:00,5,"28.0000000000000000, 21.7500000000000000, 20.0...",Haloperidol\nHaloperidol\nHaloperidol\nHaloper...,2,HALOPERIDOL\nOLANZAPINE,1,1,1,0,0,0,0,0,0,2157-05-01 05:20:41,2157-05-01 05:19:00,2157-07-01 10:20:00,NaN
5,223,M,67.51,WHITE,105694,231395,4.0629,0389: Septicemia NOS\n41021: AMI inferolateral...,3,33,2157-06-01 15:55:00,1,11.5000000000000000,Haloperidol\nHaloperidol\nHaloperidol\nHaloper...,2,HALOPERIDOL\nOLANZAPINE,1,1,1,0,0,0,0,0,0,2157-06-01 11:33:51,2157-05-01 05:19:00,2157-07-01 10:20:00,NaN


# Explore other icd9 codes

In [18]:
# data[data.icd9_codes.str.contains("518")]
all_icd_codes = pd.DataFrame(columns = ["shortcode", "longcode", "name", "count"])
for i in data.index:
    codes = data.icd9_codes[i].split("\n")
    for c in codes:
        shortcode = c[0:3]
        longcode = c.split(":")[0]
        name = c.split(": ")[1]
        ind_for_data_entry = all_icd_codes[all_icd_codes.longcode == longcode]
        if (len(ind_for_data_entry) == 0):
            all_icd_codes = pd.concat([all_icd_codes, pd.DataFrame(data={"shortcode": shortcode, "longcode":longcode, "name":name, "count":1}, index=[0])], axis=0, ignore_index=True)
        else:
            all_icd_codes.at[ind_for_data_entry.index[0],"count"] += 1

In [19]:
all_icd_codes.shape

(2422, 4)

In [20]:
# pd.set_option('display.max_colwidth', None)

- https://www.findacode.com/search/search.php
- http://www.icd9data.com/2015/Volume1/

- Other diseases (icd9 codes) *
- Surgery *
- Pneumonia *
- Infection
- Sepsis (diagnosis at admission) *
- Dementia *
- Alzheimer's *
- Depression *
- Anxiety *
- (COMPLEX) Coma (drug induced, miscellaneous, combination) [use name.str search with coma]
- Metabolic acidosis * [276.2]
- (COMPLEX) Respiratory failure [518.51, 518.53, 518.81, 581.83, 581.84, 799.1]
- (COMPLEX) ARDS (diagnosis at admission) [its complex since requires many other data, https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0257056]
- Airway Protection * (diagnosis at admission) [496] 
- [490-496] Chronic Obstructive Pulmonary Disease, [493] Asthma, or [496] other pulmonary disorder (diagnosis at admission)
 - COPD * [490-496]: a group of diseases that cause airflow blockage and breathing-related problems
- [428] Chronic heart failure, [410] myocardial infarction, or [427] AF and arrhythmia (diagnosis at admission)
 - heart issues *
- [571] Cirrhosis or [570] Liver failure (diagnosis at admission)
 - Liver issues *
- Other (diagnosis at admission)

In [21]:
data[data.icd9_codes.str.contains("496: Chr airway obstruct NEC")].shape

(175, 29)

In [22]:
all_icd_codes[all_icd_codes.name.str.contains("sepsis")]

,shortcode,longcode,name,count
27,995,99592,Severe sepsis,305


In [23]:
data['met_acidosis'] = np.where(data.icd9_codes.str.contains("2764: Mixed acid-base bal dis"), 1, 0)
data['airway_obs'] = np.where(data.icd9_codes.str.contains("496: Chr airway obstruct NEC"), 1, 0)

copd_codes = ['490: Bronchitis NOS', '49121: Obs chr bronc w(ac) exac', 
              '4928: Emphysema NEC', '49392: Asthma NOS w (ac) exac', 
              '4941: Bronchiectasis w ac exac', '4957: "ventilation" pneumonit', 
              '496: Chr airway obstruct NEC']
data['copd'] = np.where(data.icd9_codes.str.contains('|'.join(copd_codes)), 1, 0)

liver_codes = ['570: Acute necrosis of liver', '5715: Cirrhosis of liver NOS']
data['liver_disease'] = np.where(data.icd9_codes.str.contains('|'.join(liver_codes)), 1, 0)

heart_codes = []
heart_icd_codes = all_icd_codes[all_icd_codes.shortcode.str.contains("410|427|428")]
for i in heart_icd_codes.index:
    heart_codes.append(heart_icd_codes.longcode[i] + ": " + heart_icd_codes.name[i])
print(heart_codes)
data['heart_disease'] = np.where(data.icd9_codes.str.contains('|'.join(heart_codes)), 1, 0)

['42731: Atrial fibrillation', '4280: CHF NOS', '41071: Subendo infarct, initial', '41021: AMI inferolateral, init', '42821: Ac systolic hrt failure', '42781: Sinoatrial node dysfunct', '4271: Parox ventric tachycard', '41051: AMI lateral NEC, initial', '42741: Ventricular fibrillation', '42789: Cardiac dysrhythmias NEC', '42823: Ac on chr syst hrt fail', '42820: Systolic hrt failure NOS', '42830: Diastolc hrt failure NOS', '41041: AMI inferior wall, init', '42822: Chr systolic hrt failure', '41092: AMI NOS, subsequent', '4275: Cardiac arrest', '42843: Ac/chr syst/dia hrt fail', '41091: AMI NOS, initial', '42732: Atrial flutter', '41011: AMI anterior wall, init', '41072: Subendo infarct, subseq', '42832: Chr diastolic hrt fail', '42842: Chr syst/diastl hrt fail', '42833: Ac on chr diast hrt fail', '42841: Ac syst/diastol hrt fail', '41032: AMI inferopost, subseq', '42840: Syst/diast hrt fail NOS', '42831: Ac diastolic hrt failure', '41081: AMI NEC, initial', '4270: Parox atrial tachyca

/var/folders/k5/t6qrq2v55gxd80p9mdn45x9r8q8tz2/T/ipykernel_22719/3113583476.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['copd'] = np.where(data.icd9_codes.str.contains('|'.join(copd_codes)), 1, 0)


In [24]:
sorted_codes = all_icd_codes.sort_values('count', ascending=False)
sorted_codes.head(31)

,shortcode,longcode,name,count
9,293,2930,Delirium d/t other cond,1671
67,401,4019,Hypertension NOS,708
2,427,42731,Atrial fibrillation,599
3,428,4280,CHF NOS,571
23,518,51881,Acute respiratry failure,434
134,599,5990,Urin tract infection NOS,433
15,584,5849,Acute kidney failure NOS,418
90,272,2724,Hyperlipidemia NEC/NOS,374
30,414,41401,Crnry athrscl natve vssl,356
114,250,25000,DMII wo cmp nt st uncntr,310


# Formatting the datasets for use in exploratory, ml and causal analysis

In [25]:
from collections import Counter

In [26]:
def label_drug_name(row, frequency):
    drugs_list = row['drugs'].lower().split('\n')
    drugs_list_counter = Counter(drugs_list)
    if (frequency == 0):
        value, count = drugs_list_counter.most_common()[0]
    elif (frequency == 1):
        if (len(drugs_list_counter) > 1):
            value, count = drugs_list_counter.most_common()[1]
        else:
            value = 'NA'
    else:
        raise AttributeError("Frequency must be 0 or 1")
    return value

In [27]:
def label_drug_type(row):
    if "no drug" in row['drug_1']:
        return 'ND'
    if "haloperidol" in row['drug_1']:
        return 'HL'
    if "ziprasidone" in row['drug_1']:
        return 'ZP'
    return 'OD'

In [28]:
def label_timelength(row):
    if (pd.isna(row['drugstarttime'])):
        return -1
    
    if (pd.notna(row["deathtime"])):
        final_time = row["deathtime"]
    else:
        final_time = row["dischtime"]
            
    length = pd.Timedelta(pd.to_datetime(final_time) - pd.to_datetime(row["drugstarttime"])).total_seconds()
    length_in_days = round(length/ 86400.0) # in days
    return length_in_days

In [29]:
def label_age(row):
    if row['age'] > 299:
        return 90
    else:
        return row['age']
    return row['age']

In [30]:
# def label_sex(row):
#     if row['gender'] == 'F':
#         return 1
#     else:
#         return 0
#     return 1

In [31]:
# https://grants.nih.gov/grants/guide/notice-files/not-od-15-089.html
def label_ethnicity(row):
    if (('WHITE' in row["ethnicity"].upper()) | ('EASTERN' in row["ethnicity"].upper())):
        return 1
    if ('BLACK' in row["ethnicity"].upper()):
        return 2
    if (('HISPANIC' in row["ethnicity"].upper()) | ('LATINO' in row["ethnicity"].upper())):
        return 3
    if ('ASIAN' in row["ethnicity"].upper()):
        return 4
    if (('AMERICAN' in row["ethnicity"].upper()) | ('ALASKA' in row["ethnicity"].upper())):
        return 5
    return 0

In [32]:
def label_time_to_mechvent(row):
    if ((isinstance(row["mechvent_duration_hours"], int)) | ((isinstance(row["mechvent_duration_hours"], float)))):
        return float(row["mechvent_duration_hours"])
    else:
        if ("," in row["mechvent_duration_hours"]):
            return float(max(row["mechvent_duration_hours"].split(",")))
        else:
            return float(row["mechvent_duration_hours"])
    return 0

In [33]:
import datetime

def label_time_to_mechvent(row):
    if pd.isna(row['mechvent_starttime']):
        return -1.0
    else:
        datetimeFormat = '%Y-%m-%d %H:%M:%S'
        date1 = row['mechvent_starttime']
        date2 = row['admittime']
        diff = datetime.datetime.strptime(date1, datetimeFormat) - datetime.datetime.strptime(date2, datetimeFormat)
        return (diff.total_seconds() / 86400)
    return -1.0

In [34]:
def label_mechvent(row):
    if row['mechvent_count'] == 0:
        return 0
    else:
        return 1
    return 0

In [35]:
def label_mechvent_hours(row):
    if ((isinstance(row["mechvent_duration_hours"], int)) | ((isinstance(row["mechvent_duration_hours"], float)))):
        return float(row["mechvent_duration_hours"])
    else:
        if ("," in row["mechvent_duration_hours"]):
            return float(max(row["mechvent_duration_hours"].split(",")))
        else:
            return float(row["mechvent_duration_hours"])
    return 0

In [36]:
def label_death_type(row):
    if (pd.isna(row['deathtime']) or pd.isna(row['deathtime'])):
        return -1
        
    datetimeFormat = '%Y-%m-%d %H:%M:%S'
    date1 = row['deathtime']
    date2 = row['dischtime']
    diff = datetime.datetime.strptime(date1, datetimeFormat) - datetime.datetime.strptime(date2, datetimeFormat)
    days = (diff.total_seconds() / 86400)
    if (days <= 30):
        return 1
    elif ((days > 30) & (days <= 90)):
        return 2
    elif ((days > 90) & (days <= 366)):
        return 3
    else:
        return 4
    return 0

In [37]:
data['drug_1'] = data.apply(lambda row: label_drug_name(row, 0), axis=1)
data['drug_2'] = data.apply(lambda row: label_drug_name(row, 1), axis=1)
data['drug_group'] = data.apply(lambda row: label_drug_type(row), axis=1)
data['drug_timelength'] = data.apply(lambda row: label_timelength(row), axis=1)
data['age'] = data.apply(lambda row: label_age(row), axis=1)
data['race'] = data.apply(lambda row: label_ethnicity(row), axis=1)
data['time_to_mechvent'] = data.apply(lambda row: label_time_to_mechvent(row), axis=1)
data['mechvent'] = data.apply(lambda row: label_mechvent(row), axis=1)
data['time_in_mechvent'] = data.apply(lambda row: label_mechvent_hours(row), axis=1)
data['death_timeline'] = data.apply(lambda row: label_death_type(row), axis=1)

In [38]:
data.shape

(1671, 43)

In [39]:
data.sample(3)

,subject_id,sex,age,ethnicity,hadm_id,icustay_id,los_days,icd9_codes,sofa,apsiii,mechvent_starttime,mechvent_count,mechvent_duration_hours,drugs,drug_categories_distinct_count,drug_categories,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,death_hosp,death_one_year,drugstarttime,admittime,dischtime,deathtime,met_acidosis,airway_obs,copd,liver_disease,heart_disease,drug_1,drug_2,drug_group,drug_timelength,race,time_to_mechvent,mechvent,time_in_mechvent,death_timeline
1591,93991,F,88.70,WHITE,125524,267747,1.1474,85206: Subarach hem-coma NOS\n2930: Delirium d...,1,40,NaN,0,NaN,Prochlorperazine\nProchlorperazine,1,PROCHLORPERAZINE,0,0,0,0,0,0,0,0,0,2144-07-01 15:24:26,2144-07-01 15:23:00,2144-07-06 15:22:00,NaN,0,0,0,0,0,prochlorperazine,NA,OD,5,1,-1.0,0,NaN,-1
1315,76479,F,52.20,WHITE,157132,229374,2.6916,"5551: Reg enteritis, lg intest\n452: Portal ve...",1,38,NaN,0,NaN,Prochlorperazine\nProchlorperazine\nProchlorpe...,2,HALOPERIDOL\nPROCHLORPERAZINE,0,0,0,0,0,0,0,0,0,2176-09-20 19:37:48,2176-09-01 11:37:00,2176-10-09 15:03:00,NaN,0,0,0,0,1,prochlorperazine,haloperidol,OD,19,1,-1.0,0,NaN,-1
1465,85842,M,67.45,WHITE,130730,200575,1.3992,5770: Acute pancreatitis\n42833: Ac on chr dia...,5,41,NaN,0,NaN,Haloperidol\nOlanzapine (Disintegrating Tablet...,2,HALOPERIDOL\nOLANZAPINE,0,0,0,0,0,1,0,0,1,2139-01-08 15:46:46,2139-01-08 15:45:00,2139-01-20 14:40:00,2139-05-16 00:00:00,0,0,0,0,1,haloperidol,olanzapine (disintegrating tablet),HL,127,1,-1.0,0,NaN,3


In [40]:
data.columns

Index(['subject_id', 'sex', 'age', 'ethnicity', 'hadm_id', 'icustay_id',
       'los_days', 'icd9_codes', 'sofa', 'apsiii', 'mechvent_starttime',
       'mechvent_count', 'mechvent_duration_hours', 'drugs',
       'drug_categories_distinct_count', 'drug_categories', 'surgery',
       'pneumonia', 'sepsis', 'dementia', 'alzheimers', 'depression',
       'anxiety', 'death_hosp', 'death_one_year', 'drugstarttime', 'admittime',
       'dischtime', 'deathtime', 'met_acidosis', 'airway_obs', 'copd',
       'liver_disease', 'heart_disease', 'drug_1', 'drug_2', 'drug_group',
       'drug_timelength', 'race', 'time_to_mechvent', 'mechvent',
       'time_in_mechvent', 'death_timeline'],
      dtype='object')

In [41]:
# dropping: 
# 'ethnicity', 'mechvent_starttime','drugstarttime', 'admittime', 'dischtime', 
# 'deathtime', 'mechvent_duration_hours', 'drugs', 'drug_categories', 'death_one_year', 'drug_1', 'drug_2'
data = data[['subject_id', 'hadm_id', 'icustay_id', 'sex', 'age', 'race', 'icd9_codes',
             'sofa', 'apsiii', 'surgery', 'pneumonia', 'sepsis', 'dementia', 'alzheimers', 'depression', 
             'anxiety', 'met_acidosis', 'airway_obs', 'copd', 'liver_disease', 'heart_disease', 
             'mechvent', 'mechvent_count', 'drug_group', 'drug_categories_distinct_count', 'drug_timelength', 
             'death_hosp', 'death_timeline', 'los_days', 'time_to_mechvent', 'time_in_mechvent']]

In [42]:
data = data.astype({"drug_group": 'category'})

In [43]:
data.shape

(1671, 31)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   subject_id                      1671 non-null   int64   
 1   hadm_id                         1671 non-null   int64   
 2   icustay_id                      1671 non-null   int64   
 3   sex                             1671 non-null   category
 4   age                             1671 non-null   float64 
 5   race                            1671 non-null   int64   
 6   icd9_codes                      1671 non-null   object  
 7   sofa                            1671 non-null   int64   
 8   apsiii                          1671 non-null   int64   
 9   surgery                         1671 non-null   int64   
 10  pneumonia                       1671 non-null   int64   
 11  sepsis                          1671 non-null   int64   
 12  dementia            

In [45]:
data.sample(5)

,subject_id,hadm_id,icustay_id,sex,age,race,icd9_codes,sofa,apsiii,surgery,pneumonia,sepsis,dementia,alzheimers,depression,anxiety,met_acidosis,airway_obs,copd,liver_disease,heart_disease,mechvent,mechvent_count,drug_group,drug_categories_distinct_count,drug_timelength,death_hosp,death_timeline,los_days,time_to_mechvent,time_in_mechvent
1614,96015,166722,293747,F,74.05,1,1970: Secondary malig neo lung\n4821: Pseudomo...,4,34,1,1,0,0,0,0,0,0,0,0,0,1,1,3,HL,3,253,0,3,15.4564,1.937500,2.000000
1505,89065,100845,286443,F,83.31,1,78659: Chest pain NEC\n42842: Chr syst/diastl ...,1,38,0,0,0,0,0,0,0,0,0,0,0,1,0,0,HL,1,2,0,-1,1.1076,-1.000000,NaN
835,46217,161623,236823,F,82.60,2,99811: Hemorrhage complic proc\n5570: Ac vasc ...,11,62,1,0,0,0,0,0,0,0,0,0,0,0,1,1,ND,0,12,0,-1,8.1460,0.093056,161.083333
1343,77308,113465,207055,M,67.13,1,2536: Neurohypophysis dis NEC\n1983: Sec mal n...,2,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,OD,1,56,0,2,1.0113,-1.000000,NaN
1369,79909,175576,219892,M,87.46,3,0389: Septicemia NOS\n78552: Septic shock\n599...,1,49,0,0,1,0,1,0,0,0,0,0,0,1,0,0,OD,1,741,0,4,1.2984,-1.000000,NaN


In [46]:
# count the na s and fix it
data.isna().sum()

subject_id                          0
hadm_id                             0
icustay_id                          0
sex                                 0
age                                 0
race                                0
icd9_codes                          0
sofa                                0
apsiii                              0
surgery                             0
pneumonia                           0
sepsis                              0
dementia                            0
alzheimers                          0
depression                          0
anxiety                             0
met_acidosis                        0
airway_obs                          0
copd                                0
liver_disease                       0
heart_disease                       0
mechvent                            0
mechvent_count                      0
drug_group                          0
drug_categories_distinct_count      0
drug_timelength                     0
death_hosp  

In [47]:
data.los_days = data.los_days.fillna(-1)
data.time_in_mechvent = data.time_in_mechvent.fillna(-1)
data.isna().sum()

subject_id                        0
hadm_id                           0
icustay_id                        0
sex                               0
age                               0
race                              0
icd9_codes                        0
sofa                              0
apsiii                            0
surgery                           0
pneumonia                         0
sepsis                            0
dementia                          0
alzheimers                        0
depression                        0
anxiety                           0
met_acidosis                      0
airway_obs                        0
copd                              0
liver_disease                     0
heart_disease                     0
mechvent                          0
mechvent_count                    0
drug_group                        0
drug_categories_distinct_count    0
drug_timelength                   0
death_hosp                        0
death_timeline              

In [48]:
data.to_csv('delirium_data_cleaned.csv', header=True, index=False)
sorted_codes.to_csv('delirium_data_icd_codes.csv', header=True, index=False)